In [1]:
from enum import Enum, auto
from typing import Optional

import numpy as n
from matplotlib import pyplot as plt 

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
class Action(Enum):
    UP = auto()
    DOWN = auto()
    UP_RIGHT = auto()
    DOWN_RIGHT = auto()
    RIGHT = auto()

ACTIONS_LIST = [a for a in Action]
ACTIONS_LIST_NO_DOWN = [a for a in ACTIONS_LIST if a not in [Action.DOWN, Action.DOWN_RIGHT]]
ACTIONS_LIST_NO_UP = [a for a in ACTIONS_LIST if a not in [Action.UP, Action.UP_RIGHT]]
N_ACTIONS = len(ACTIONS_LIST)

In [ ]:
# Proc gen constants
SEED = 32
SIDE_LEN = 100
MIN_Y = - (1/5) * SIDE_LEN
MAX_Y = abs(MIN_Y)

# Draw constants
CMAP = "gray"

In [ ]:
bg = np.zeros((SIDE_LEN, SIDE_LEN), dtype=int)
plt.imshow(bg, cmap=plt.get_cmap(CMAP))

In [ ]:
rng = np.random.default_rng(SEED)

def get_rand_action(actions_list: np.ndarray, action_probs: np.ndarray) -> int:
    return rng.choice(actions_list, p=action_probs)

def generate_tesselation_mask_side():
    mask = np.copy(bg)
    cursor = {"x": 0, "y": 0}
    last_action = None
    while cursor["x"] < side_len-1:
        action = get_rand_action(ACTIONS_LIST, action_probs)
        if action == Action.UP:
            cursor["y"] -= 1
        elif action == Action.UP_RIGHT:
            cursor["y"] -= 1
            cursor["x"] += 1
        elif action == Action.RIGHT:
            cursor["x"] += 1
        elif action == Action.DOWN:
            cursor["y"] += 1
        elif action == Action.DOWN_RIGHT:
            cursor["y"] += 1
            cursor["x"] += 1
        else:
            raise ValueError(f"Unsupported action: {action}")

        if cursor["y"] >= 0:
            mask[0:cursor["y"], cursor["x"]] = 1
        else:
            mask[cursor["y"]:, cursor["x"]] = 1
        
        last_action = action
    return mask

def generate_tesselation_mask():
    y_mask = generate_tesselation_mask_side()
    x_mask = generate_tesselation_mask_side().T
    return x_mask | y_mask
    
ACTION_PROBS = np.full(N_ACTIONS, 1 / N_ACTIONS)
mask = generate_tesselation_mask()
plt.imshow(final_mask, cmap=plt.get_cmap(CMAP))

In [ ]:
def tesselate(mask: np.ndarray, n_shapes: int = 5) -> np.ndarray:
    side_len_full_image = mask.shape[0] * n_shapes
    full_image = np.zeros((side_len_full_image, side_len_full_image), dtype=int)
    
    color = 0
    for i in range(n_shapes):
        for j in range(n_shapes):
            y_start = i * side_len
            y_end = y_start + side_len
            
            x_start = j * side_len
            x_end = x_start + side_len
    
            if color == 0:
                color_mask = np.logical_not(mask)
                color = 1
            else:
                color_mask = mask
                color = 0
            
            full_image[y_start:y_end, x_start:x_end] = color_mask
            
    return full_image
plt.imshow(tesselate(mask), cmap=plt.get_cmap(CMAP))

In [ ]:
from IPython.display import clear_output

while True:
    mask = generate_tesselation_mask()
    plt.imshow(tesselate(mask), cmap=plt.get_cmap(CMAP))
    plt.show()
    
    print("Press 'q' to quit, anything else to keep generating...")
    user_input = input()
    if user_input.lower() == "q":
        break
    clear_output()